In [1]:
#
# Data merger for CALPADS
#

import os
import pandas as pd
import re
from tqdm import tqdm

In [20]:
sheet_names = [
    'LEA-Level CALPADS UPC Data',
    'School-Level CALPADS UPC Data'
]
school_data_cols = {
    'School Code': 'school_code',
    'School Name': 'school_name',
    'School Type': 'school_type',
    'Low Grade':   'grade_low',
    'High Grade':  'grade_high',
}
calpads_data_cols = {
    'School Code':                            'school_code',
    'Academic Year':                          'academic_year',
    'Total Enrollment':                       'enrollment',
    'Free & Reduced Meal Program':            'frpm_count',
    'Foster':                                 'foster_count',
    'Tribal Foster Youth':                    'foster_tribal_count',
    'Homeless':                               'homeless_count',
    'Migrant Program':                        'migrant_program_count',
    'Direct Certification':                   'direct_certification_count',
    'Unduplicated FRPM Eligible Count':       'frpm_eligible_count',
    'English Learner (EL)':                   'english_learned_count',
    'CALPADS Unduplicated Pupil Count (UPC)': 'unduplicated_count'
}



def load_sheet ( filename, sheet_name ):
    sheet = pd.read_excel( filename, sheet_name )
    sheet.columns = [ re.sub( r'\s+', ' ', colname ) for colname in list( sheet.iloc[0] ) ]
    sheet = sheet[sheet['County Name'] == 'San Francisco'].reset_index( drop = True )

    return sheet



raw_data_directory = '../raw/'
all_school_data = pd.DataFrame()
all_calpads_data = pd.DataFrame()
for filename in tqdm( os.listdir( raw_data_directory ) ):
    full_filename = f'{raw_data_directory}{filename}'

    for sheet_name in sheet_names:
        sheet = load_sheet( full_filename, sheet_name )

        school_data = sheet[[ col for col in school_data_cols.keys() if col in sheet]].rename( columns = school_data_cols )
        calpads_data = sheet[[ col for col in calpads_data_cols.keys() if col in sheet]].rename( columns = calpads_data_cols )

        all_school_data = pd.concat(
            [
                all_school_data,
                school_data
            ],
            axis = 0,
            ignore_index = True
        ).drop_duplicates()

        all_calpads_data = pd.concat(
            [
                all_calpads_data,
                calpads_data.assign( source_sheet = sheet_name )
            ],
            axis = 0,
            ignore_index = True
        )

all_school_data = all_school_data[~pd.isnull( all_school_data.grade_low )]
all_calpads_data = all_calpads_data.merge( all_school_data[['school_code']].drop_duplicates() )

all_school_data.to_csv( './sfusd-school-codes.csv', index = False )
all_calpads_data.to_csv( './sfusd-calpads-data.csv', index = False )

  0%|          | 0/8 [00:00<?, ?it/s]

/opt/homebrew/lib/python3.11/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'CALPADS UPC Data Notes'!$2:$2.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/homebrew/lib/python3.11/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'CALPADS UPC Data Notes'!$2:$2.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
 12%|█▎        | 1/8 [00:01<00:10,  1.45s/it]/opt/homebrew/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
 25%|██▌       | 2/8 [00:02<00:08,  1.40s/it]/opt/homebrew/lib/pyth